# 6일차 3.11 (Installing ESLint)

- const videos = await Video.find({}).sort({ '_id': -1 }); // -1 은 반대로 정렬한다는 의미
- eslint : 기본적으론 변수가 존재하지 않는데도 에러로 표시를 안해줌. eslint 깔면 도와줌.
- eslint --init : eslint 초기설정

    √ How would you like to use ESLint? · style       
    
    √ What type of modules does your project use? · esm
    
    √ Which framework does your project use? · none
    
    √ Does your project use TypeScript? · No / Yes
    
    √ Where does your code run? · browser
    
    √ How would you like to define a style for your project? · guide
    
    √ Which style guide do you want to follow? · airbnb
    
    √ What format do you want your config file to be in? · JavaScript
    

- npm install eslint, eslint-config-prettier, eslint-plugin-prettier, prettier
- eslilnt에서는 console.log를 쓰지말라고함. 이 때 eslintrc.js 에서 rules: { "no-console": "off" } 로 경고 없앨 수 있음.

# 7일차 3.12 (Searching Videos)

- model.find()과 정규식 표현을 같이 사용할 수 있음
- pug 에서 if .. else 문 사용법, 

In [ ]:
// controllers/videoController.js

// ... 중략 ...

videos = await Video.find({ title: { $regex: searchingBy, $options: "i" } })

// ... 중략 ...

In [ ]:
// views/videoDetails.pug

extends layouts/main.pug

block content
	.video__player
		video(src=video.fileUrl, width=300, height=300, controls=true)
	.video__info
		a(href=routes.editVideo(video.id)) Edit Video
		h5.video__title=video.title
		span.video__views=video.views
		p.video__description=video.description
	.video__comments
		if video.comments.length === 1
			span.video__comment-number 1 comment
		else
			span.video__comment-number #{video.comments.length} comments

# 7일차 4.0 (Introduction to Webpack)

- Backend 50% 끝내고, Frontend 시작한다고 함
- SCSS 개념 (https://heropy.blog/2018/01/31/sass/)
- webpack이란? module bundler
- npm install webpack webpack-cli
- webpack-cli는 터미널에서 사용할 수 있도록 해줌
- webpack.config.js 생성해야함
- webpack은 entry와 output이 있어야 함
- assets/js/main.js, assets/scss/styles.scss 만듦


In [ ]:
// webpack.config.js

const path = require("path");

const ENTRY_FILE = path.resolve(__dirname, "assets", "js", "main.js");
const OUTPUT_DIR = path.join(__dirname, "static");

const config = {
	entry: ENTRY_FILE,
	output: {
		path: OUTPUT_DIR,
		filename: "[name].[format]"
	}
}

module.exports = config;

In [ ]:
// main.js

import "../scss/styles.scss";


In [ ]:
// styles.css

body {
	background-color: red;
}

# 7일차 4.1 (Styles with Webpack part One)

- webpack Warning 경고 뜨는 거 제거 하기

In [ ]:
// package.json

  "scripts": {
    "dev:server": "nodemon --exec babel-node init.js --delay 2",
    "dev:assets": "WEBPACK_ENV=development webpack", // 환경 변수 추가함과 동시에 webpack 실행
    "build:assets": "WEBPACK_ENV=production webpack"
  }

// webpack.config.js

const path = require("path");

const MODE = process.env.WEBPACK_ENV; // 환경 변수에서 불러옴
const ENTRY_FILE = path.resolve(__dirname, "assets", "js", "main.js");
const OUTPUT_DIR = path.join(__dirname, "static");

const config = {
	entry: ENTRY_FILE,
	mode: MODE,
	output: {
		path: OUTPUT_DIR,
		filename: "[name].[format]"
	}
}

module.exports = config;

- npm install extract-text-webpack-plugin@next
- webpack은 아래에서부터 위로 코드를 작성해야한다

# 7일차 4.2 (Continued...)

- npm install css-loader postcss-loader sass-loader
- sass-loader가 sass파일을 css 로 변환해주고,
- postcss-loader가 브라우저별로 css의 호환성을 보장해주고 (prefix들을 자동으로 붙여줌. ex: -webkit-, -moz-, 등등)
- css-loader가 css를 텍스트로 반환해서 우리가 쓰도록 해준다.
- npm install autoprefixer
- npm install node-sass
- npm install cross-env // 윈도우에서 환경변수 설정하기 위한 패키지 설치함

In [ ]:
// package.json

"scripts": {
    "dev:server": "nodemon --exec babel-node init.js --delay 2",
    "dev:assets": "cross-env WEBPACK_ENV=development webpack", // cross platform 에서는 cross-env 사용
    "build:assets": "cross-env WEBPACK_ENV=production webpack"
  },
      
// webpack.config.js
      
const path = require("path");
const autoprefixer = require("autoprefixer");
const ExtractCSS = require("extract-text-webpack-plugin");

const MODE = process.env.WEBPACK_ENV;
const ENTRY_FILE = path.resolve(__dirname, "assets", "js", "main.js");
const OUTPUT_DIR = path.join(__dirname, "static");

const config = {
	entry: ["@babel/polyfill", ENTRY_FILE],
	mode: MODE,
	module: {
		rules: [
			{
				test: /\.(js)$/,
				use: [
					{
						loader: "babel-loader"
					}
				]
			},
			{
				test: /\.(scss)$/,
				use: ExtractCSS.extract([
					{
						loader: "css-loader" // css 를 text로 반환
					},
					{
						loader: "postcss-loader", // 호환성
						options: {
							plugins() {
								return [autoprefixer({ overrideBrowserslist: "cover 99.5%" })]; // 호환성
							}
						}
					},
					{
						loader: "sass-loader" // sass to css converison
					}
				])
			}
		]
	},
	output: {
		path: OUTPUT_DIR,
		filename: "[name].js"
	},
	plugins: [new ExtractCSS("styles.css")]
};

module.exports = config;

// assets/scss/styles.scss

@import "/config/_variables.scss";
body {
	background-color: $bgColor;
}

// assets/scss/config/_variables.scss

$bgColor: red; // 이와 같이 변수를 선언할 수 있고, 위처럼 사용할 수 있다.